In [1]:
import pandas as pd

df = pd.read_csv("/home/liang/Desktop/python_file/source.csv")

In [2]:
df = df[["poemId", "poemTitle", "poemContent"]]
df

,poemId,poemTitle,poemContent
0,1,关雎,关关雎鸠，在河之洲。窈窕淑女，君子好逑。\n参差荇菜，左右流之。窈窕淑女，寤寐求之。\n求之...
1,2,葛覃,葛之覃兮，施于中谷，维叶萋萋。黄鸟于飞，集于灌木，其鸣喈喈。\n葛之覃兮，施于中谷，维叶莫莫...
2,3,卷耳,采采卷耳，不盈顷筐。嗟我怀人，寘彼周行。\n陟彼崔嵬，我马虺隤。我姑酌彼金罍，维以不永怀。\...
3,4,樛木,南有樛木，葛藟累之。乐只君子，福履绥之。\n南有樛木，葛藟荒之。乐只君子，福履将之。\n南有...
4,5,螽斯,螽斯羽，诜诜兮。宜尔子孙，振振兮。\n螽斯羽，薨薨兮。宜尔子孙，绳绳兮。\n螽斯羽，揖揖兮。...
...,...,...,...
72406,73277,题八咏楼,千古风流八咏楼，江山留与后人愁。\n水通南国三千里，气压江城十四州。
72407,73278,偶成,十五年前花月底，相从曾赋赏花诗。\n今看花月浑相似，安得情怀似往时。
72408,73279,江行,暝色蒹葭外，苍茫旅眺情。\n残雪和雁断，新月带潮生。\n天到水中尽，舟随树杪行。\n离家今几...
72409,73280,芙蓉池作,乘辇夜行游，逍遥步西园。\n双渠相溉灌，嘉木绕通川。\n卑枝拂羽盖，修条摩苍天。\n惊风扶轮...


In [3]:
from pyspark.sql import SQLContext, SparkSession
import numpy as np
import os

os.environ['JAVA_HOME'] = "/usr/local/src/jdk1.8.0_172"
os.environ["SPARK_HOME"] = "/usr/local/src/spark-2.2.0-bin-hadoop2.6"
os.environ["PYTHONPATH"] = "/home/liang/miniconda3/bin/python"

spark = SparkSession.builder.appName("abc").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

values = df.values.tolist()
columns = df.columns.tolist()
item_info = spark.createDataFrame(values, columns)
item_info.show()

+------+---------+--------------------+
|poemId|poemTitle|         poemContent|
+------+---------+--------------------+
|     1|       关雎|关关雎鸠，在河之洲。窈窕淑女，君子...|
|     2|       葛覃|葛之覃兮，施于中谷，维叶萋萋。黄鸟...|
|     3|       卷耳|采采卷耳，不盈顷筐。嗟我怀人，寘彼...|
|     4|       樛木|南有樛木，葛藟累之。乐只君子，福履...|
|     5|       螽斯|螽斯羽，诜诜兮。宜尔子孙，振振兮。...|
|     6|       桃夭|桃之夭夭，灼灼其华。之子于归，宜其...|
|     7|       兔罝|肃肃兔罝，椓之丁丁。赳赳武夫，公侯...|
|     8|       芣苢|采采芣苢，薄言采之。
采采芣苢，薄...|
|     9|       汉广|南有乔木，不可休思。
汉有游女，不...|
|    10|       汝坟|遵彼汝坟，伐其条枚。未见君子，惄如...|
|    11|      麟之趾|麟之趾，振振公子，于嗟麟兮。
麟之...|
|    12|       鹊巢|维鹊有巢，维鸠居之。之子于归，百两...|
|    13|       采蘩|于以采蘩？于沼于沚。于以用之？公侯...|
|    14|       草虫|喓喓草虫，趯趯阜螽。未见君子，忧心...|
|    15|       采蘋|于以采蘋？南涧之滨。于以采藻？于彼...|
|    16|       甘棠|蔽芾甘棠，勿翦勿伐，召伯所茇。
蔽...|
|    17|       行露|厌浥行露，岂不夙夜，谓行多露。
谁...|
|    18|       羔羊|羔羊之皮，素丝五紽。退食自公，委蛇...|
|    19|      殷其雷|殷其雷，在南山之阳。何斯违斯，莫敢...|
|    20|      摽有梅|摽有梅，其实七兮。求我庶士，迨其吉...|
+------+---------+--------------------+
only showing top 20 rows



In [4]:
from pyspark.sql.functions import concat_ws

# 合并古诗的标题和正文
sentence_df = item_info.select("poemId", 
    concat_ws(",", 
              item_info.poemTitle,
              item_info.poemContent,
             ).alias("concat_string")
)
sentence_df.show()

+------+--------------------+
|poemId|       concat_string|
+------+--------------------+
|     1|关雎,关关雎鸠，在河之洲。窈窕淑女...|
|     2|葛覃,葛之覃兮，施于中谷，维叶萋萋...|
|     3|卷耳,采采卷耳，不盈顷筐。嗟我怀人...|
|     4|樛木,南有樛木，葛藟累之。乐只君子...|
|     5|螽斯,螽斯羽，诜诜兮。宜尔子孙，振...|
|     6|桃夭,桃之夭夭，灼灼其华。之子于归...|
|     7|兔罝,肃肃兔罝，椓之丁丁。赳赳武夫...|
|     8|芣苢,采采芣苢，薄言采之。
采采芣...|
|     9|汉广,南有乔木，不可休思。
汉有游...|
|    10|汝坟,遵彼汝坟，伐其条枚。未见君子...|
|    11|麟之趾,麟之趾，振振公子，于嗟麟兮...|
|    12|鹊巢,维鹊有巢，维鸠居之。之子于归...|
|    13|采蘩,于以采蘩？于沼于沚。于以用之...|
|    14|草虫,喓喓草虫，趯趯阜螽。未见君子...|
|    15|采蘋,于以采蘋？南涧之滨。于以采藻...|
|    16|甘棠,蔽芾甘棠，勿翦勿伐，召伯所茇...|
|    17|行露,厌浥行露，岂不夙夜，谓行多露...|
|    18|羔羊,羔羊之皮，素丝五紽。退食自公...|
|    19|殷其雷,殷其雷，在南山之阳。何斯违...|
|    20|摽有梅,摽有梅，其实七兮。求我庶士...|
+------+--------------------+
only showing top 20 rows



In [5]:
import re
import pyspark.sql.functions as F
from pyspark.sql.types import *

# 替换里边的特殊字符
def _filter(arg):
    arg = re.sub('[\n\r（）。，、？！,]', '', arg)
    return arg

use_reg = F.udf(_filter, StringType())
sentence_df = sentence_df.select(sentence_df.poemId, \
                                     use_reg(sentence_df.concat_string).alias("all_words"))

sentence_df.show()

+------+--------------------+
|poemId|           all_words|
+------+--------------------+
|     1|关雎关关雎鸠在河之洲窈窕淑女君子好...|
|     2|葛覃葛之覃兮施于中谷维叶萋萋黄鸟于...|
|     3|卷耳采采卷耳不盈顷筐嗟我怀人寘彼周...|
|     4|樛木南有樛木葛藟累之乐只君子福履绥...|
|     5|螽斯螽斯羽诜诜兮宜尔子孙振振兮螽斯...|
|     6|桃夭桃之夭夭灼灼其华之子于归宜其室...|
|     7|兔罝肃肃兔罝椓之丁丁赳赳武夫公侯干...|
|     8|芣苢采采芣苢薄言采之采采芣苢薄言有...|
|     9|汉广南有乔木不可休思汉有游女不可求...|
|    10|汝坟遵彼汝坟伐其条枚未见君子惄如调...|
|    11|麟之趾麟之趾振振公子于嗟麟兮麟之定...|
|    12|鹊巢维鹊有巢维鸠居之之子于归百两御...|
|    13|采蘩于以采蘩于沼于沚于以用之公侯之...|
|    14|草虫喓喓草虫趯趯阜螽未见君子忧心忡...|
|    15|采蘋于以采蘋南涧之滨于以采藻于彼行...|
|    16|甘棠蔽芾甘棠勿翦勿伐召伯所茇蔽芾甘...|
|    17|行露厌浥行露岂不夙夜谓行多露谁谓雀...|
|    18|羔羊羔羊之皮素丝五紽退食自公委蛇委...|
|    19|殷其雷殷其雷在南山之阳何斯违斯莫敢...|
|    20|摽有梅摽有梅其实七兮求我庶士迨其吉...|
+------+--------------------+
only showing top 20 rows



In [6]:
import jieba

In [7]:
def get_words(partitions):
    stop_words = ['而', '何', '乎', '乃', '其', '且', '若', '所', '为', '焉', '以', 
              '因', '于', '与','也','则','者','之','不','自','得','一','来','去',
              '无', '可', '是', '已', '此', '的', '上', '中', '兮', '三', '汝', '非']
    
    def cut_sentence(sentence):
        return [i for i in jieba.cut(sentence, cut_all=True) if i not in stop_words]
    
    for row in partitions:
        yield row.poemId, cut_sentence(row.all_words)
        
sentence_df = sentence_df.rdd.mapPartitions(get_words).toDF(["poemId", "word_list"])
sentence_df.show()

+------+--------------------+
|poemId|           word_list|
+------+--------------------+
|     1|[关, 雎, 关关, 关关雎, 雎...|
|     2|[葛, 覃, 葛, 覃, 施, 中...|
|     3|[卷, 耳, 采采, 卷, 耳, ...|
|     4|[樛, 木, 南, 有, 樛, 木...|
|     5|[螽斯, 螽斯, 羽, 诜, 诜,...|
|     6|[桃, 夭, 桃之夭夭, 灼灼, ...|
|     7|[兔, 罝, 肃, 肃, 兔, 罝...|
|     8|[芣, 苢, 采采, 芣, 苢, ...|
|     9|[汉, 广南, 有, 乔木, 不可...|
|    10|[坟, 遵, 彼, 坟, 伐, 条...|
|    11|[麟, 趾, 麟, 趾, 振振, ...|
|    12|[鹊巢, 维, 鹊, 有, 巢, ...|
|    13|[采, 蘩, 采, 蘩, 沼, 沚...|
|    14|[草虫, 喓, 喓, 草虫, 趯,...|
|    15|[采, 蘋, 采, 蘋, 南涧, ...|
|    16|[甘, 棠, 蔽, 芾, 甘, 棠...|
|    17|[行, 露, 厌, 浥, 行, 露...|
|    18|[羔羊, 羊羔, 羔羊, 皮, 素...|
|    19|[殷, 雷, 殷, 雷, 在, 南...|
|    20|[摽, 有, 梅, 摽, 有, 梅...|
+------+--------------------+
only showing top 20 rows



In [8]:
# 要计算TFIDF值，需要知道每个词的词频和反文档频率
from pyspark.ml.feature import CountVectorizer


# 统计词频
cv = CountVectorizer(inputCol="word_list", outputCol="word_frequency", minDF=1.0)

cv_model = cv.fit(sentence_df)
cv_result= cv_model.transform(sentence_df)
cv_result.show()

+------+--------------------+--------------------+
|poemId|           word_list|      word_frequency|
+------+--------------------+--------------------+
|     1|[关, 雎, 关关, 关关雎, 雎...|(69204,[9,94,262,...|
|     2|[葛, 覃, 葛, 覃, 施, 中...|(69204,[7,13,53,7...|
|     3|[卷, 耳, 采采, 卷, 耳, ...|(69204,[2,5,13,56...|
|     4|[樛, 木, 南, 有, 樛, 木...|(69204,[3,46,91,1...|
|     5|[螽斯, 螽斯, 羽, 诜, 诜,...|(69204,[408,460,5...|
|     6|[桃, 夭, 桃之夭夭, 灼灼, ...|(69204,[3,7,120,1...|
|     7|[兔, 罝, 肃, 肃, 兔, 罝...|(69204,[100,1170,...|
|     8|[芣, 苢, 采采, 芣, 苢, ...|(69204,[3,594,550...|
|     9|[汉, 广南, 有, 乔木, 不可...|(69204,[0,1,3,7,9...|
|    10|[坟, 遵, 彼, 坟, 伐, 条...|(69204,[13,15,18,...|
|    11|[麟, 趾, 麟, 趾, 振振, ...|(69204,[137,506,6...|
|    12|[鹊巢, 维, 鹊, 有, 巢, ...|(69204,[3,7,46,12...|
|    13|[采, 蘩, 采, 蘩, 沼, 沚...|(69204,[7,28,242,...|
|    14|[草虫, 喓, 喓, 草虫, 趯,...|(69204,[15,22,108...|
|    15|[采, 蘋, 采, 蘋, 南涧, ...|(69204,[3,14,61,6...|
|    16|[甘, 棠, 蔽, 芾, 甘, 棠...|(69204,[798,1016,...|
|    17|[行, 露, 厌, 浥, 行, 露...|(6

In [9]:
print(len(cv_model.vocabulary))
cv_model.vocabulary

69204


['',
 ' ',
 '人',
 '有',
 '春',
 '云',
 '花',
 '归',
 '月',
 '在',
 '君',
 '时',
 '风',
 '我',
 '谁',
 '见',
 '日',
 '玉',
 '如',
 '山',
 '秋',
 '尽',
 '未',
 '香',
 '空',
 '更',
 '寒',
 '作',
 '还',
 '和',
 '欲',
 '送',
 '清',
 '愁',
 '看',
 '客',
 '多',
 '醉',
 '酒',
 '长',
 '寄',
 '雨',
 '天',
 '似',
 '生',
 '新',
 '将',
 '梦',
 '水',
 '知',
 '应',
 '远',
 '又',
 '飞',
 '入',
 '处',
 '金',
 '歌',
 '相',
 '烟',
 '向',
 '行',
 '闲',
 '前',
 '别',
 '雪',
 '下',
 '闻',
 '夜',
 '高',
 '红',
 '莫',
 '过',
 '道',
 '老',
 '心',
 '犹',
 '深',
 '\u3000',
 '满',
 '里',
 '年',
 '同',
 '翠',
 '望',
 '初',
 '难',
 '诗',
 '朝',
 '到',
 '开',
 '成',
 '树',
 '事',
 '思',
 '笑',
 '轻',
 '声',
 '独',
 '书',
 '好',
 '重',
 '游',
 '问',
 '路',
 '出',
 '晓',
 '明',
 '亦',
 '回',
 '柳',
 '共',
 '对',
 '吟',
 '复',
 '仙',
 '却',
 '后',
 '落',
 '江',
 '子',
 '孤',
 '题',
 '今',
 '青',
 '起',
 '恨',
 '竹',
 '从',
 '露',
 '白',
 '休',
 '小',
 '忆',
 '尘',
 '旧',
 '晚',
 '公',
 '绿',
 '身',
 '随',
 '王',
 '赠',
 '霜',
 '林',
 '曾',
 '寻',
 '草',
 '家',
 '情',
 '能',
 '衣',
 '芳',
 '听',
 '·',
 '碧',
 '溪',
 '倚',
 '地',
 '梅',
 '断',
 '坐',
 '暮',
 '正',
 '影',
 '松',


In [10]:
from pyspark.ml.feature import IDF
idf = IDF(inputCol="word_frequency", outputCol="IDF_value")

idfModel = idf.fit(cv_result)
rescaledData = idfModel.transform(cv_result)

rescaledData.select("word_list", "IDF_value").show()

+--------------------+--------------------+
|           word_list|           IDF_value|
+--------------------+--------------------+
|[关, 雎, 关关, 关关雎, 雎...|(69204,[9,94,262,...|
|[葛, 覃, 葛, 覃, 施, 中...|(69204,[7,13,53,7...|
|[卷, 耳, 采采, 卷, 耳, ...|(69204,[2,5,13,56...|
|[樛, 木, 南, 有, 樛, 木...|(69204,[3,46,91,1...|
|[螽斯, 螽斯, 羽, 诜, 诜,...|(69204,[408,460,5...|
|[桃, 夭, 桃之夭夭, 灼灼, ...|(69204,[3,7,120,1...|
|[兔, 罝, 肃, 肃, 兔, 罝...|(69204,[100,1170,...|
|[芣, 苢, 采采, 芣, 苢, ...|(69204,[3,594,550...|
|[汉, 广南, 有, 乔木, 不可...|(69204,[0,1,3,7,9...|
|[坟, 遵, 彼, 坟, 伐, 条...|(69204,[13,15,18,...|
|[麟, 趾, 麟, 趾, 振振, ...|(69204,[137,506,6...|
|[鹊巢, 维, 鹊, 有, 巢, ...|(69204,[3,7,46,12...|
|[采, 蘩, 采, 蘩, 沼, 沚...|(69204,[7,28,242,...|
|[草虫, 喓, 喓, 草虫, 趯,...|(69204,[15,22,108...|
|[采, 蘋, 采, 蘋, 南涧, ...|(69204,[3,14,61,6...|
|[甘, 棠, 蔽, 芾, 甘, 棠...|(69204,[798,1016,...|
|[行, 露, 厌, 浥, 行, 露...|(69204,[13,14,36,...|
|[羔羊, 羊羔, 羔羊, 皮, 素...|(69204,[137,474,7...|
|[殷, 雷, 殷, 雷, 在, 南...|(69204,[7,9,55,71...|
|[摽, 有, 梅, 摽, 有, 梅...|(69204,[3,

In [11]:
idfModel.idf.toArray()

array([ 0.15462493,  0.16263591,  1.66914546, ..., 10.49698013,
       10.49698013, 10.49698013])

In [13]:
from functools import partial

def _tfidf(partition, kw_list):
    for row in partition:
        words_length = len(set(row.word_list))    # 统计文档中单词总数

        for index in row.word_frequency.indices:
            word, idf = kw_list[int(index)] 
            tf = row.word_frequency[int(index)]/words_length   # 计算TF值
            tfidf = float(tf)*float(idf)    # 计算该词的TFIDF值
            yield row.poemId, word, tfidf

# 使用partial为函数预定义要传入的参数
tfidf = partial(_tfidf, kw_list=keywords_list_with_idf)            

keyword_tfidf = cv_result.rdd.mapPartitions(tfidf)
keyword_tfidf = keyword_tfidf.toDF(["poemId","keyword", "tfidf"])
keyword_tfidf.show()

+------+-------+--------------------+
|poemId|keyword|               tfidf|
+------+-------+--------------------+
|     1|      在|0.057601030605564936|
|     1|      思| 0.07921524240863861|
|     1|      流| 0.09759119429122505|
|     1|      关| 0.10977667106706135|
|     1|      采|  0.1198127377805513|
|     1|     不得| 0.11853473276288638|
|     1|     君子| 0.14131315456663138|
|     1|     参差| 0.40177176553544663|
|     1|      友|  0.1353286374743755|
|     1|      服| 0.13801581262352733|
|     1|     窈窕|  0.6544337720537399|
|     1|     左右|  0.4968428369265834|
|     1|     钟鼓|  0.1849036696914497|
|     1|     悠哉|  0.3774377046757293|
|     1|      菜|  0.5753820425157967|
|     1|      荇|  0.5824866990705768|
|     1|     琴瑟|  0.2073800479363288|
|     1|     关关| 0.21292736263464188|
|     1|     寤寐| 0.45325890128424867|
|     1|      雎| 0.24643365580097992|
+------+-------+--------------------+
only showing top 20 rows



In [12]:
keywords_list_with_idf = list(zip(cv_model.vocabulary, idfModel.idf.toArray()))
keywords_list_with_idf

[('', 0.15462493099127095),
 (' ', 0.16263591402670433),
 ('人', 1.6691454619329604),
 ('有', 1.799717916778991),
 ('春', 1.7968822184651163),
 ('云', 1.8183485926421212),
 ('花', 1.9757949172501144),
 ('归', 1.966772001442157),
 ('月', 1.9429729653969354),
 ('在', 2.016036071194773),
 ('君', 2.165393886305136),
 ('时', 2.0749771255234006),
 ('风', 2.066434745245324),
 ('我', 2.344206077191815),
 ('谁', 2.1310750527334332),
 ('见', 2.1562854820108175),
 ('日', 2.1792139632165455),
 ('玉', 2.2088228155386993),
 ('如', 2.2794066485444757),
 ('山', 2.2291461366522736),
 ('秋', 2.252252045166572),
 ('尽', 2.233647462495922),
 ('未', 2.2525147191896218),
 ('香', 2.2666693308008683),
 ('空', 2.247666383675253),
 ('更', 2.25593577515776),
 ('寒', 2.2576507020340943),
 ('作', 2.303856408423822),
 ('还', 2.277654036029798),
 ('和', 2.3537533795684453),
 ('欲', 2.3376053931604632),
 ('送', 2.3520107128480143),
 ('清', 2.3481007809970236),
 ('愁', 2.366332161775305),
 ('看', 2.3434864973953484),
 ('客', 2.3635394797221663),
 ('多'

In [14]:
keyword_tfidf.orderBy("tfidf", ascending=False).show()

+------+-------+------------------+
|poemId|keyword|             tfidf|
+------+-------+------------------+
|     8|      苢| 7.347886090955122|
|     8|      芣| 6.706482578643214|
|     8|     薄言|5.1751067718205785|
| 46406|      耶| 4.948944803640301|
|    36|     式微| 4.750594433822543|
|     8|     采采| 4.598260071527804|
|    84|      萚|4.4562877042617925|
| 47827|     阿房| 4.178456983219809|
|   610|     没了| 4.037300049975342|
|    11|      麟| 3.801857875632987|
|    48|      彊| 3.759347136507112|
|    18|     委蛇| 3.759347136507112|
|     5|     螽斯| 3.483887053840631|
| 46611|     丹徒| 3.450071181213719|
| 46555|     举子| 3.367015435302422|
| 46616|     徐闻|3.2679443164586477|
| 47675|    段干木|3.1935631326872445|
|    48|      奔|3.1468240211335554|
| 65160|      囗| 3.132789280422593|
| 70885|     莲叶| 3.114219717233316|
+------+-------+------------------+
only showing top 20 rows



In [15]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=10, inputCol="word_list", outputCol="model")
model = word2Vec.fit(sentence_df)

In [16]:
vectors = model.getVectors()
vectors.show()

+----+--------------------+
|word|              vector|
+----+--------------------+
| 半日闲|[-0.0561875551939...|
|  箭头|[0.12245123833417...|
|  临风|[-0.2168009430170...|
|  琴书|[-0.2475093156099...|
|   邱|[0.05485768243670...|
|  莲社|[0.09266380965709...|
|  人物|[-0.0869999676942...|
|  长幼|[0.18391117453575...|
|   冉|[0.60794192552566...|
|  石道|[0.09541463851928...|
|   婺|[0.52122080326080...|
|  泻出|[-0.0349056571722...|
|  黄泉|[-0.0684615746140...|
|  本源|[-0.0193176493048...|
|  自养|[0.06587809324264...|
|  吾国|[-0.0718164145946...|
|  命作|[0.02833748422563...|
|  相怜|[1.22657394967973...|
|  疏狂|[-0.0754516199231...|
|   溱|[0.28383380174636...|
+----+--------------------+
only showing top 20 rows



In [17]:
# 合并
df1 = keyword_tfidf.join(vectors, keyword_tfidf.keyword==vectors.word, "inner")
df1.show()

+------+-------+--------------------+----+--------------------+
|poemId|keyword|               tfidf|word|              vector|
+------+-------+--------------------+----+--------------------+
|     1|      在|0.057601030605564936|   在|[-0.0991405770182...|
|     1|      思| 0.07921524240863861|   思|[-0.2970317900180...|
|     1|      流| 0.09759119429122505|   流|[-0.0167482625693...|
|     1|      关| 0.10977667106706135|   关|[0.00378159806132...|
|     1|      采|  0.1198127377805513|   采|[-0.0207617487758...|
|     1|     不得| 0.11853473276288638|  不得|[-0.0765498951077...|
|     1|     君子| 0.14131315456663138|  君子|[-0.0510748177766...|
|     1|     参差| 0.40177176553544663|  参差|[0.17438517510890...|
|     1|      友|  0.1353286374743755|   友|[-0.1573343873023...|
|     1|      服| 0.13801581262352733|   服|[0.43713137507438...|
|     1|     窈窕|  0.6544337720537399|  窈窕|[0.03826018050312...|
|     1|     左右|  0.4968428369265834|  左右|[0.15486374497413...|
|     1|     钟鼓|  0.1849036696914497|  钟

In [18]:
# print(keyword_tfidf.count())
# print(df1.count())

In [19]:
# 使用每个词的权重乘以没个词的词向量
df2 = df1.rdd.map(lambda r:(r.poemId, r.keyword, r.tfidf*r.vector)).toDF(["poemId", "keyword","vector"])
df2.show()

+------+-------+--------------------+
|poemId|keyword|              vector|
+------+-------+--------------------+
|     1|      在|[-0.0057105994110...|
|     1|      思|[-0.0235294452493...|
|     1|      流|[-0.0016344829464...|
|     1|      关|[4.15131246485710...|
|     1|      采|[-0.0024875219619...|
|     1|     不得|[-0.0090738213596...|
|     1|     君子|[-0.0072175436189...|
|     1|     参差|[0.07006303968671...|
|     1|      友|[-0.0212918482614...|
|     1|      服|[0.06033104195413...|
|     1|     窈窕|[0.02503875424612...|
|     1|     左右|[0.07694294239002...|
|     1|     钟鼓|[0.02371931855677...|
|     1|     悠哉|[-0.1086785448600...|
|     1|      菜|[0.17224957892647...|
|     1|      荇|[0.28912690197140...|
|     1|     琴瑟|[0.02254813662401...|
|     1|     关关|[0.00427683555109...|
|     1|     寤寐|[0.10319108292020...|
|     1|      雎|[0.00167664982013...|
+------+-------+--------------------+
only showing top 20 rows



In [20]:
# 分组求出加权词向量的平均值
# 创建临时表
df2.registerTempTable("tempTable")

def map(row):
    x = 0
    for v in row.vectors:
        x += v
    #  将平均向量作为sku的向量
    return row.poemId, x/len(row.vectors)

group_vector = spark.sql("select poemId, collect_set(vector) vectors from tempTable group by poemId").rdd.map(map).toDF(["poemId", "vector"])

In [21]:
# group_vector.count()

In [22]:
# 计算两首古诗之间的余弦相似度
v1 = group_vector.where("poemId=1").select("vector").first().vector
v2 = group_vector.where("poemId=2").select("vector").first().vector

np.dot(v1,v2)/(np.linalg.norm(v1)*(np.linalg.norm(v2)))

0.378789818447398

In [23]:
group_df = group_vector.toPandas()

In [24]:
group_df

,poemId,vector
0,26,"[0.008604375373175262, 0.025260454274981203, -..."
1,29,"[0.017804960879639398, 0.04290195911035433, 0...."
2,474,"[0.008394777557195702, 0.025892256004034084, 0..."
3,964,"[4.312995210865666e-05, 0.04748191617755371, -..."
4,1677,"[0.014743320294977227, -0.031115929525233794, ..."
...,...,...
72406,73102,"[0.007128699103552195, 0.008001016583454229, 0..."
72407,73148,"[0.018791418906944357, -0.026507278938189598, ..."
72408,73179,"[-0.002067103872106531, -0.021090942436392375,..."
72409,73240,"[-0.005216105655755288, -0.018684123660531675,..."


In [25]:
group_df.to_pickle("./word_vector_10hidden.pk")

In [26]:
group_df["vector"][0].dot(group_df["vector"][1])

0.004892947822319301

In [27]:
def cosine_similarity(v1, v2):
    return np.dot(v1,v2)/(np.linalg.norm(v1)*(np.linalg.norm(v2)))

# for i in range(1, 73282):
#     v1 = group_df[group_df["poemId"] == 1]["vector"].values[0]
#     for j in range(1, 73282):
#         if i != j:
            
# cosine_similarity(group_df[group_df["poemId"] == 1]["vector"].values[0], group_df[group_df["poemId"] == 2]["vector"].values[0])

In [28]:
group_df[group_df["poemId"] == 1]["vector"].values[0]

DenseVector([0.0302, 0.032, -0.0349, 0.0162, -0.0161, -0.0174, -0.0619, 0.0057, -0.0074, 0.0124])

In [29]:
group_df["poemId"].max()

73281

In [30]:
group_df[group_df["poemId"] == 49]["vector"].values

array([], dtype=object)

In [31]:
none_list = []

for i in range(1, 73282):
    if not group_df[group_df["poemId"] == i]["vector"].values:
        none_list.append(i)

/home/liang/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  after removing the cwd from sys.path.


In [32]:
exist_id = [i for i in range(1, 73282) if i not in none_list]

In [33]:
len(exist_id), len(none_list)

(72411, 870)

In [34]:
fsadfsdaf

NameError: name 'fsadfsdaf' is not defined

In [ ]:
del tqdm

In [ ]:
from tqdm import tqdm
import random

content_base_dict = {}

# 现根据标签筛选一部分
for i in tqdm(exist_id):
    v1 = group_df[group_df["poemId"] == i]["vector"].values[0]
#     random_id = random.choices(exist_id, k=1000)改成从标签中筛选
    store_list = []
    for j in random_id:
        if i != j:
            v2 = group_df[group_df["poemId"] == j]["vector"].values[0]
            sim = cosine_similarity(v1, v2)
            store_list.append((j, sim))
    value = sorted(store_list, key=lambda x:x[1], reverse=True)[:100]
    content_base_dict[i] = value
    break

In [ ]:
sorted(store_list, key=lambda x:x[1], reverse=True)

In [ ]:
group_df[group_df["poemId"] == 30000]["vector"].values[0]

In [35]:
one_hot_df = pd.read_pickle("/home/liang/Desktop/python_file/one_hot_df.pk")
one_hot_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72411 entries, 0 to 72410
Columns: 1032 entries, poemId to 魏晋
dtypes: float64(1), int64(1015), object(1), uint8(15)
memory usage: 562.9+ MB


In [40]:
one_hot_df

,poemId,poemDynasty,poemAuthorId,poemStar,七夕节,丈夫,不满,不舍,中秋,中秋节,...,唐代,宋代,明代,未知,清代,现代,近代,金朝,隋代,魏晋
0,1,先秦,200000,14233.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,先秦,200000,1869.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,先秦,200000,1037.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,先秦,200000,684.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,先秦,200000,464.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72406,73277,宋代,200536,16.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
72407,73278,宋代,200536,19.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
72408,73279,宋代,200196,13.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
72409,73280,魏晋,200616,5.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [42]:
# mongodb里面存的诗人id是在原来基础上加了10w
one_hot_df["poemAuthorId"]

0        200000
1        200000
2        200000
3        200000
4        200000
          ...  
72406    200536
72407    200536
72408    200196
72409    200616
72410    200272
Name: poemAuthorId, Length: 72411, dtype: int64

In [47]:
one_hot_df[one_hot_df["poemId"] == 1]['七夕节']

0    0
Name: 七夕节, dtype: int64

In [49]:
poem_to_tag = pd.read_csv("/home/liang/Desktop/python_file/source.csv")
poem_to_tag = poem_to_tag[["poemId", "poemDynasty", "poemAuthorId", "poemTagNames"]]
poem_to_tag

,poemId,poemDynasty,poemAuthorId,poemTagNames
0,1,先秦,0,"古诗三百首,初中古诗,诗经,爱情"
1,2,先秦,0,"诗经,写人"
2,3,先秦,0,"诗经,怀人"
3,4,先秦,0,"诗经,祝福"
4,5,先秦,0,"诗经,写鸟,祝福"
...,...,...,...,...
72406,73277,宋代,536,"歌颂,古人,伤怀,国家"
72407,73278,宋代,536,"抒情,追忆,思念"
72408,73279,宋代,196,"羁旅,抒情,思乡"
72409,73280,魏晋,616,"荷花,写景,夜晚,心情"


In [51]:
poem_to_tag["poemAuthorId"] += 100000
poem_to_tag

,poemId,poemDynasty,poemAuthorId,poemTagNames
0,1,先秦,100000,"古诗三百首,初中古诗,诗经,爱情"
1,2,先秦,100000,"诗经,写人"
2,3,先秦,100000,"诗经,怀人"
3,4,先秦,100000,"诗经,祝福"
4,5,先秦,100000,"诗经,写鸟,祝福"
...,...,...,...,...
72406,73277,宋代,100536,"歌颂,古人,伤怀,国家"
72407,73278,宋代,100536,"抒情,追忆,思念"
72408,73279,宋代,100196,"羁旅,抒情,思乡"
72409,73280,魏晋,100616,"荷花,写景,夜晚,心情"


In [82]:
import pymongo

client = pymongo.MongoClient("localhost", 27017)
mycol = client['RS']
houxuanji = mycol['online_recall']

In [ ]:
from tqdm import tqdm
import random


content_base_dict = {}

def findFromMongoDB(collection, tag):
    temp_set = set()
    query_dict = {
        "cateId":tag,
    }
    found_json = collection.find(query_dict)
    for _json in found_json:
        items = _json.get("incluted")
        items_len = _json.get("len")
        if items_len < 100: # 每个类别随机选取100个
            pass
        else:
            items = random.choices(items, k=100)
        temp_set = temp_set.union(items)
        
    return temp_set

# 现根据标签筛选一部分
for i in tqdm(exist_id):
    v1 = group_df[group_df["poemId"] == i]["vector"].values[0]
    # 古诗id -> 古诗标签 -> 根据标签进行筛选
    temp = poem_to_tag[poem_to_tag["poemId"] == i]
    tags = temp["poemTagNames"].values[0]
    poemId_set = set()
    if tags is not np.nan:
        tags = tags.split(',')
        for tag in tags:
            ids = findFromMongoDB(houxuanji, tag)
            poemId_set = poemId_set.union(ids)
    
    dynasty = temp["poemDynasty"].values[0]
    author = temp["poemAuthorId"].values[0]
    ids1 = findFromMongoDB(houxuanji, dynasty)
    ids2 = findFromMongoDB(houxuanji, int(author))
    poemId_set = poemId_set.union(ids1)
    poemId_set = poemId_set.union(ids2)
    
    print("length:",len(poemId_set))
    store_list = []
    for j in poemId_set:
        if i != j:
            v2 = group_df[group_df["poemId"] == j]["vector"].values[0]
            sim = cosine_similarity(v1, v2)
            store_list.append((j, sim))
            
    value = sorted(store_list, key=lambda x:x[1], reverse=True)[:100]
    content_base_dict[i] = [tup[0] for tup in value]

In [63]:
poem_to_tag[poem_to_tag["poemId"] == 1]["poemTagNames"].values[0]

'古诗三百首,初中古诗,诗经,爱情'

In [98]:
content_base_dict

{1: [48013,
  48005,
  42372,
  606,
  21,
  70919,
  6525,
  69012,
  48107,
  47991,
  15,
  47642,
  47682,
  10521,
  72453,
  47981,
  34,
  47873,
  70886,
  69221,
  127,
  45792,
  70933,
  137,
  7734,
  69077,
  10374,
  71030,
  2369,
  47685,
  47692,
  42412,
  2271,
  70978,
  7,
  68055,
  68709,
  20285,
  69086,
  47708,
  69068,
  48147,
  9849,
  47728,
  71708,
  9098,
  78,
  71043,
  30793,
  39490,
  69053,
  47813,
  42180,
  71002,
  70940,
  70897,
  9315,
  172,
  69071,
  6606,
  47044,
  70999,
  21173,
  39469,
  30,
  223,
  59,
  67634,
  16,
  69055,
  45951,
  5749,
  151,
  47696,
  24030,
  71203,
  213,
  71780,
  48134,
  150,
  69088,
  62369,
  71205,
  69067,
  159,
  21137,
  58,
  47733,
  59106,
  47966,
  52809,
  67872,
  111,
  6137,
  70765,
  69142,
  71646,
  48041,
  67880,
  4933]}

In [102]:
poem_to_tag[poem_to_tag["poemId"] == 306]

,poemId,poemDynasty,poemAuthorId,poemTagNames
304,306,唐代,100582,NaN


In [100]:
i

306

In [103]:
if poem_to_tag[poem_to_tag["poemId"] == 306]['poemTagNames'].values[0] is not pd.NA:
    print(1)

1


In [107]:
poem_to_tag[poem_to_tag["poemId"] == 306]['poemTagNames'].values[0] is np.nan

True

In [109]:
import pickle

In [111]:
pickle.dump(content_base_dict, open('content_base_dict.pk', 'wb'))

In [9]:
import pickle

In [10]:
content_base_dict = pickle.load(open('content_base_dict.pk', 'rb'))

In [3]:
content_base_dict[1]

[42465,
 71038,
 48013,
 18023,
 48005,
 47977,
 70919,
 6525,
 180,
 47695,
 48107,
 47809,
 70896,
 10521,
 18,
 47981,
 71004,
 47873,
 70886,
 99,
 69221,
 127,
 70894,
 24472,
 137,
 71007,
 71016,
 41948,
 142,
 47688,
 10374,
 2369,
 134,
 47712,
 22497,
 21744,
 46455,
 7790,
 67142,
 7,
 6419,
 46499,
 71026,
 20285,
 48003,
 89,
 68115,
 71031,
 71863,
 69068,
 48147,
 70994,
 143,
 5453,
 42369,
 52,
 70932,
 47995,
 9098,
 126,
 42461,
 71043,
 28810,
 106,
 69053,
 68008,
 47989,
 71036,
 70949,
 70940,
 4,
 70897,
 9315,
 115,
 45955,
 69071,
 6606,
 28798,
 47697,
 67993,
 171,
 21173,
 30,
 138,
 223,
 71396,
 16,
 71277,
 64769,
 5749,
 70906,
 70964,
 71203,
 48134,
 150,
 69088,
 71205,
 159,
 21137,
 52809]

In [11]:
# 将计算得到的数据导入到 mongodb
import pymongo

In [12]:
client = pymongo.MongoClient("localhost", 27017)
mycol = client['RS']
houxuanji = mycol['Word2Vec_sim']

for key, value in content_base_dict.items():
    temp_dict = {"poemId":key, "top100":value}
    houxuanji.insert_one(temp_dict)

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused

In [13]:
# 导入到服务器的 mongodb
client = pymongo.MongoClient("120.26.89.198", 27017)
mycol = client['Recommendation']
houxuanji = mycol['Word2Vec_sim']

# counter = 0
# ll = []
# for key, value in content_base_dict.items():
#     counter += 1
#     temp_dict = {"poemId":key, "top100":value}
#     ll.append(temp_dict)
#     if counter % 100 == 0:
#         print(counter)
#         houxuanji.insert_many(ll)
#         ll = []
        

for i in range(73271,73282):
    temp_dict = {"poemId":i, "top100":content_base_dict[i]}
    houxuanji.insert_one(temp_dict)

In [14]:
c = houxuanji.find({"poemId":70000})
for i in c:
    print(i)

In [19]:
content_base_dict[73282]

KeyError: 73282